Есть гипотеза:

Если мы применим новую модель рекомендаций, то повысим продажи продуктов `deposit_4` и `insurance`.

Решили проверить данное утверждение через AB-тест, который проводился с 2021-02-24 по 2021-03-12.

В clients залили клиентов с group_type=’tg’ - у кого новая рекомендательная система; ‘cg’ - у кого рекомендации оставили без изменений.

Твоя задача: 
1. Рассчитать результаты ab-теста, сделать репрезентативные визуализации (python, excel, power bi - любой удобный тебе) + таблички и вывод о том, подтвердилась ли гипотеза + пояснить вывод.
2. Представь что паркеты с данными это таблички в RDBMS и напиши sql-запрос, который покажет % изменения конверсии по продуктам `deposit_4` и `insurance` между cg и tg на период теста + используй оператор with, для читаемости.

Данные:

Таблицы:

1. сlients – список клиентов:
* client_id - id клиента
* group_type – к какой группе относится (tg-целевая, cg-тестовая)
2. shows – показы продуктов клиентам:
* calday – дата показа
* product_id – какой продукт показали
3. sales – продажи продуктов:
* calday – дата продажи
* product_id – какой продукт продали
4. products – каталог продуктов:
* id – id продукта
* product_name – наименование продукта


Читаются через pandas.read_parquet

In [100]:
import pandas as pd
import numpy as np
import seaborn as sns

from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))


In [101]:
clients = pd.read_parquet('clients.parquet')
shows = pd.read_parquet('shows.parquet')
sales = pd.read_parquet('sales.parquet')
products = pd.read_parquet('products.parquet')

In [102]:
shows.isnull().sum()

client_id        0
calday           0
product_id    3521
dtype: int64

In [103]:
sales.isnull().sum()

client_id        0
calday           0
product_id    3521
dtype: int64

In [104]:
products.isnull().sum()

id              0
product_name    0
dtype: int64

In [105]:
clients

,client_id,group_type
0,346650,tg
1,731109,tg
2,348929,tg
3,492187,tg
4,1062172,tg
...,...,...
393023,344940,tg
393024,1056210,tg
393025,1060062,tg
393026,171181,tg


In [106]:
clients.group_type.value_counts()

cg    196682
tg    196346
Name: group_type, dtype: int64

In [107]:
show_clients = shows.merge(clients, how='left', on='client_id')

In [109]:
sales_products = sales.merge(products, how='left', left_on='product_id', right_on='id')
sales_products = sales.merge(clients, how='left', on='client_id')

In [110]:
sales_products

,client_id,calday,product_id,group_type
0,163024,2021-02-10,645.0,cg
1,762783,2021-02-10,376.0,tg
2,741073,2021-02-10,987.0,tg
3,992292,2021-02-10,987.0,tg
4,854649,2021-02-10,442.0,tg
...,...,...,...,...
43224,721480,2021-04-01,442.0,cg
43225,569055,2021-04-01,645.0,cg
43226,482458,2021-04-01,154.0,tg
43227,164227,2021-04-01,154.0,cg


deposit_4 и insurance

Решили проверить данное утверждение через AB-тест, который проводился с 2021-02-24 по 2021-03-12

In [118]:
sales_pdoructs.query('product_name == "deposit_4" or product_name == "insurance"').product_name.value_counts()

insurance    5799
deposit_4    5714
Name: product_name, dtype: int64

In [117]:
sales_products_before = sales_pdoructs[sales_products.calday < '2021-02-24'].query('product_name == "deposit_4" or product_name == "insurance"')

UndefinedVariableError: name 'product_name' is not defined

In [113]:
counts_sales_products_before = sales_products_before.product_name.value_counts()
counts_sales_products_before

insurance    1589
deposit_4    1567
Name: product_name, dtype: int64

In [114]:
sales_products_ab = sales_pdoructs[(sales_products.calday > '2021-02-24') & (sales_products.calday < '2021-03-12')].query('product_name == "deposit_4" or product_name == "insurance"')

In [115]:
sales_products_ab.product_name.value_counts()

insurance    1698
deposit_4    1690
Name: product_name, dtype: int64

In [116]:
sales_products_ab

,client_id,calday,product_id,product_name
11879,993722,2021-02-25,645.0,insurance
11881,618098,2021-02-25,987.0,deposit_4
11887,311033,2021-02-25,645.0,insurance
11889,551632,2021-02-25,645.0,insurance
11891,699761,2021-02-25,987.0,deposit_4
...,...,...,...,...
26765,388419,2021-03-11,987.0,deposit_4
26769,331527,2021-03-11,645.0,insurance
26776,973567,2021-03-11,987.0,deposit_4
26779,906601,2021-03-11,645.0,insurance
